# RAG on blog post
##### https://aeon.co/essays/how-did-america-become-the-nation-of-credit-cards

In [16]:
%%capture
!pip install -q -U torch transformers accelerate bitsandbytes langchain langchain-community langchain-huggingface sentence-transformers faiss-gpu openpyxl pacmap datasets  ragatouille

In [17]:
import datasets
from IPython.display import Markdown, display
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_huggingface import HuggingFaceEmbeddings
from ragatouille import RAGPretrainedModel
import torch
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, Pipeline
from typing import Optional, List, Tuple

In [18]:
EMBEDDING_MODEL_NAME = "thenlper/gte-large"
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
RERANKER_NAME = "colbert-ir/colbertv2.0"

### Load the knowledge base
##### Manually extracted from website

In [19]:
ds = datasets.load_dataset("csv", data_files="data/blog-post.txt", split="train")

### Create the embedding model

In [20]:
def split_documents(chunk_size: int, knowledge_base: List[LangchainDocument], tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME) -> List[LangchainDocument]:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
    )
    docs = (d for doc in knowledge_base for d in text_splitter.split_documents([doc]))
    unique_contents = {doc.page_content: doc for doc in docs}
    return list(unique_contents.values())

RAW_KNOWLEDGE_BASE = [ LangchainDocument(page_content=doc["text"]) for doc in tqdm(ds) ]
docs_processed = split_documents(512, RAW_KNOWLEDGE_BASE, tokenizer_name=EMBEDDING_MODEL_NAME)

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE)

  0%|          | 0/63 [00:00<?, ?it/s]

### Create the pipeline

In [21]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
READER_LLM = pipeline(model=model, tokenizer=tokenizer, task="text-generation", do_sample=True, temperature=0.2, repetition_penalty=1.1, return_full_text=False, max_new_tokens=500)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [22]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)
RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_NAME)

In [23]:
def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)
    answer = llm(final_prompt)[0]["generated_text"]
    
    return answer, relevant_docs

def answer(question):
    a, relevant_docs = answer_with_rag(question, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)
    print("QUESTION:")
    display(Markdown("**" + question + "**" ))
    print("ANSWER:")
    display(Markdown("**" + a + "**" ))
    print("_" * 50)
    print("Source docs:")
    for i, doc in enumerate(relevant_docs):
        print(f"\tDocument {i}:")
        print("\t\t" + doc)

### Asking questions!

In [24]:
answer("Which businesses made credit cards central to their postwar plans?")

100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.71it/s]


QUESTION:


**Which businesses made credit cards central to their postwar plans?**

ANSWER:


**Gasoline companies, railroads, department stores, and the Diners Club (introduced by Frank McNamara) made credit cards central to their postwar plans, as described in Documents 0, 1, 2, and 4, respectively. Other sources in the context may provide additional examples, but these are the ones specifically mentioned.**

__________________________________________________
Source docs:
	Document 0:
		Other businesses also made credit cards central to their postwar plans. Gasoline companies, like Standard Oil of New Jersey, had developed nationwide charge account networks linking service stations in the years before the war. Wartime rationing halted credit sales. But in the late 1940s, service stations heavily promoted gasoline credit cards. Railroads, too, rolled out unified, card-based credit plans.
	Document 1:
		The postwar growth of department store chains provided an opportunity. Department stores offered credit cards. Small retailers did not. In the early 1950s, a cohort of bankers in cities and towns across the country began experimenting with local card plans that linked small retailers into local credit networks. Although the plans were modest, bankers saw opportunity. Banks ‘should be the reservoirs for every type of credit in their communities,’ a Virginia banker observed in 1953, predicting t

In [25]:
answer("Why did bankers move into the credit card market?")

100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 42.69it/s]


QUESTION:


**Why did bankers move into the credit card market?**

ANSWER:


**Bankers moved into the credit card market in the late 1960s as a way to escape the tight regulations that constrained their industry during the New Deal era. Credit cards provided a way for them to innovate around these restrictions and build consumer lending markets from scratch, particularly in affluent, white suburbs where traditional banking services were becoming less accessible due to demographic changes. However, the initial enthusiasm for credit cards as a source of profits turned into disappointment as cards became another source of low-margin lending due to a consumer backlash and legislation that followed. Despite this setback, bankers continued to pursue credit cards as a means of recruiting new customers and expanding their businesses beyond traditional geographic territories.**

__________________________________________________
Source docs:
	Document 0:
		Bankers leapt into the credit card market in the late 1960s to escape the tight regulations that constrained their industry. The consumer backlash and the wave of legislation that followed fundamentally constrained credit card plans. Instead of a source of profits, cards became another source of low-margin lending. What had once seemed like the road to the future appeared as just another dead end.
	Document 1:
		The cascade of unsolicited credit drew the attention of consumer groups, labour unions and policymakers at all levels of government. Bankers pursued cards to innovate around New Deal regulatory restrictions and to build consumer lending markets from scratch. Many did so to expand beyond downtowns that were becoming Blacker and poorer, and to reach affluent, white suburbs. In doing so, bankers inadvertently sparked a political backlash that would drastically hem in their ambitions.
	Document 2:
		The 

In [26]:
answer("What did Frank McNamara introduced and what was it used for?")

100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 44.32it/s]


QUESTION:


**What did Frank McNamara introduced and what was it used for?**

ANSWER:


**Frank McNamara introduced the Diners Club card in 1950, which allowed executives to pay for meals and entertainment at restaurants and clubs in New York City and later around the country. This credit card marked a watershed moment in the development of universal travel-and-entertainment cards, as it went beyond store-specific department store cards and gasoline and rail cards that linked independent businesses within the travel industry under a unified credit plan. The Diners Club card paved the way for the expansion of credit cards into the full suite of travel and entertainment expenses. (Source: Document 0)**

__________________________________________________
Source docs:
	Document 0:
		These travel cards set the stage for ‘universal’ travel-and-entertainment cards. Department store cards, offered by firms like Macy’s or Gimbels, were store specific. Gasoline and rail cards linked independent businesses within the travel industry under a unified credit plan. The watershed came in 1950, when Frank McNamara introduced the Diners Club card to executives in New York City. The name was self-explanatory. The card allowed executives to wine and dine clients at restaurants and clubs, first in New York and soon around the country. The plan quickly expanded to include the full suite of travel and entertainment
	Document 1:
		Those national priorities changed course when the nation shifted from recovery to warmaking during the Second World War. Policymakers wanted consumers to save, not spend, a policy the US Federal Reserve pursued through firm controls on consumer credit. Government controls encoura